In [1]:
!pip install pytube
!pip install opencv-python

You should consider upgrading via the 'C:\Users\JunHyeok\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JunHyeok\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
from pytube import YouTube
import os
import glob
import cv2


file_dir = os.getcwd()
txt_list = glob.glob(os.path.join(file_dir, 'list.txt'))

In [2]:
os.getcwd()

'c:\\Users\\JunHyeok\\Desktop\\cvl\\youtube2image'

In [3]:
#영상 다운로더 1

for txt in txt_list:
    file_name = os.path.basename(txt)
    cls = file_name.split(".")[0]
    
    save_dir = os.path.join(file_dir,"src")
    
    if os.path.exists(save_dir) == False:
        os.mkdir(save_dir)

    print(f"Downloading videos is started")
    
    with open(txt, "r") as f:
        urllines = f.readlines()

        for idx ,url in enumerate(urllines):
            yt = YouTube(url)
            try:
                print(f"Video {yt.title} is downloading...")
                stream = yt.streams.filter(only_video=True).order_by('resolution').desc().first() #가장 화질 좋도록
                stream.download(save_dir)
            except:
                print(">>> Something is wrong, stop downloading current file...\n")
            
            
        print(f"< Downloading videos is finished >\n")

print("Done!")

Video [4K HDR] 경기도 성남 판교신도시 야간드라이빙 2 (판교테크노밸리~판교원마을~산운마을) #Seongnam #Pangyo #Gyeonggi #Driving #Korea is downloading...
>>> Something is wrong, stop downloading current file...

Video [4K HDR] 경기도 북부 평화로 야간드라이빙 (의정부~양주덕정) #NightDriving #Uijeongbu #Yangju #Gyeonggi #Korea #Pyunghwa-ro is downloading...
>>> Something is wrong, stop downloading current file...

Video [4K] Long Beach🇺🇸 Night Drive - Driving Downtown - Long Beach, California is downloading...
>>> Something is wrong, stop downloading current file...

Video Hollywood 4K - Night Drive is downloading...
Video [4K] Los Angeles🇺🇸 Night Drive - Sunset Boulevard - Driving Downtown is downloading...


In [1]:
#영상 다운로더 2 (하나하나 해야됨)

from pytube import YouTube
from pytube import Stream
from tqdm import tqdm

def progress_callback(stream: Stream, data_chunk: bytes, bytes_remaining: int) -> None:    
    pbar.update(len(data_chunk))


yt = YouTube('https://youtu.be/q-V4knoQ_4Y'           #여기다 url
             ,on_progress_callback=progress_callback)

yt_streams=yt.streams

for info, stream in enumerate(yt_streams):
    print(info,'영상 정보 ==>> ',stream)


itag = input('itag 숫자 선택해서 int 넣어주세요\n2160p(4k) 중에서 프레임 낮고, vcodec숫자 낮은걸로(hdr인데 용량차이), type=video로 : ')
# https://gist.github.com/sidneys/7095afe4da4ae58694d128b1034e01e2 에서 itag 숫자 참고해주세요. 2160p

my_stream = yt_streams.get_by_itag(itag)
print(my_stream,'다운합니다')
pbar = tqdm(total=my_stream.filesize,unit="bytes")

my_stream.download()
pbar.close()

print('done')

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
def extract_frame(file_dir ,file_name):
    
    filepath = os.path.join(file_dir,"src", file_name)
    video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함
    
    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)


    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)
    
    frame_dir = os.path.join(file_dir,"frame")
    frame_path = os.path.join(frame_dir, file_name[:-4])
    
    try:
        if not os.path.exists(frame_path):
            os.makedirs(frame_path)
            
    except OSError:
        print ('Error: Creating directory. ' +  frame_path)

    count = 0
    success, image = video.read()
    print("Read a new frame : ", success)
        
    while success:
            print('Saved frame number :', count)
            cv2.imwrite(frame_path + "/frame%d.jpg" % count, image)
            video.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) #1초 마다 추출
            success, image = video.read()
            print("Read a new frame : ", success)
            count += 1

    video.release()

In [ ]:
filepath = os.path.join(file_dir, "src")
file_list = []

for file in os.listdir(filepath):
    if file.endswith((".webm",".mp4")):
        file_list.append(file)

for file in file_list:
    extract_frame(file_dir, file)